In [1]:
! pip install -r requirements.txt

  Using cached packaging-24.2-py3-none-any.whl.metadata (3.2 kB)
Using cached packaging-24.2-py3-none-any.whl (65 kB)
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0


# Tutorial for Project 2

## Outline
1. Prerequisites
2. Clone the GitHub Repository and Install Dependencies
3. User Agent
4. Tool-calling DB Agent using GPT-4o-mini
5. Running Inference
6. Viewing the Conversation Trajectories
7. Submission

## Prerequisites

### Set OpenAI API key

In [4]:
import getpass
from IPython.display import clear_output

clear_output()
# Please enter your API key
new_api_key = ''
while len(new_api_key) == 0:
    new_api_key = getpass.getpass("Please input your API key: ")
    clear_output()

### Clone the GitHub Repository and Install Dependencies

In [ ]:
%cd /content
!rm -rf ai612_project_2

In [ ]:
# Cloning the GitHub repository
!git clone -q https://github.com/benchay1999/ai612_project_2.git
%cd ai612_project_2

# Installing dependencies
! pip install -r -q requirements.txt

## User Agent

Where it is, how it is implemented


## Baseline: Tool-calling Agent

### Tools

what tools we are going to use, how to define them, where to save them, how to call them

### Running

shell scripts

## What you need to do:
1. Implement your own agent in `src/agents/` (`TODO_implement_agent.py`)
2. Implement the tools your agent will use in `src/envs/mimic_iv/tools/` (`TODO_implement_tool.py`)
3. Change `src/envs/mimic_iv/env.py` to update the tools the DB agent can use in the environment.
4. Change `src/agent_factory.py` to make it able to call your implemented agent.

## Viewing Conversation Trajectories

## Submission